# Verifying "Analogies Explained" via Synthetic Data

This notebook verifies the paper by constructing synthetic co-occurrence data where paraphrases hold by design.

**Core equation:** For $w^* = W$ (e.g., king = {man, royalty}):

$$\text{PMI}(c, w^*) = \sum_{w \in W} \text{PMI}(c, w) + \rho(c) + \sigma(c) - \tau$$

In [1]:
import numpy as np
import random
from collections import defaultdict

**Data Generation.** We sample windows so that `king` and `{man, royalty}` share identical context distributions, enforcing paraphrase error $\rho \approx 0$.

In [2]:
def generate_data(n_windows=100_000, x=1/6, seed=42, scale=10):
    np.random.seed(seed)
    random.seed(seed)
    
    NEUTRAL = [f"neutral_{i}" for i in range(10 * scale)]
    MAN = [f"man_{i}" for i in range(4 * scale)]
    WOMAN = [f"woman_{i}" for i in range(4 * scale)]
    ROYALTY = [f"royalty_{i}" for i in range(2 * scale)]
    
    CONTEXTS = {
        'man': MAN + NEUTRAL,
        'woman': WOMAN + NEUTRAL,
        'king': MAN + ROYALTY,
        'queen': WOMAN + ROYALTY,
        '<royalty>': ROYALTY,
        ('<royalty>', 'man'): MAN + ROYALTY,
        ('<royalty>', 'woman'): WOMAN + ROYALTY,
    }
    TARGETS = ['man', 'woman', 'king', 'queen', '<royalty>']
    
    counts = defaultdict(int)
    paired_counts = defaultdict(int)
    target_conditioned_counts = defaultdict(lambda: defaultdict(int))
    
    for _ in range(n_windows):        
        if random.random() < 5 * x:
            targets = [random.choice(TARGETS)]
            ctx_key = targets[0]
        else:
            targets = random.choice([['man', '<royalty>'], ['woman', '<royalty>']])
            ctx_key = tuple(sorted(targets))
            paired_counts[ctx_key] += 1
        
        context = list(set(random.sample(CONTEXTS[ctx_key], k=4)))
        
        for w0 in targets:
            counts[w0] += 1
            for w1 in context:
                paired_counts[tuple(sorted((w0, w1)))] += 1
        
        for i, w1 in enumerate(context):
            counts[w1] += 1
            target_conditioned_counts[ctx_key][w1] += 1
            for j in range(i+1, len(context)):
                w2 = context[j]
                paired_counts[tuple(sorted((w1, w2)))] += 1
                
    return counts, paired_counts, target_conditioned_counts

**PMI and Embeddings**


Laplace smoothing ($\alpha$) prevents log(0) but introduces bias. Different probability estimates use different normalizations:

For $p(c|w)$ from co-occurrence matrix:
$$p(c|w) = \frac{\alpha}{N + \alpha n}$$

For $p(c|W)$ from triple counts (pair has ~3 context slots):
$$p(c|W) = \frac{0.5\alpha}{(p/2) \cdot N + 0.5\alpha n}$$

Using inconsistent smoothing inflates the residual because many tokens accumulate small biases.

In [3]:
def build_all(counts, paired_counts, target_conditioned_counts, total, alpha=1, rank=50):
    vocab = sorted(counts.keys())
    word2idx = {w: i for i, w in enumerate(vocab)}
    n = len(vocab)
    
    p_w = np.array([(counts[w] + alpha) / (total + alpha * n) for w in vocab])
    
    p_cw = np.zeros((n, n))
    for (w1, w2), cnt in paired_counts.items():
        i, j = word2idx[w1], word2idx[w2]
        p_cw[i, j] = cnt
        p_cw[j, i] = cnt
    p_cw = (p_cw + alpha) / (total + alpha * n)
    
    PMI = np.log(p_cw / (p_w[:, None] * p_w[None, :]))
    
    PAIRS = [('<royalty>', 'man'), ('<royalty>', 'woman')]
    p_W = {pair: paired_counts[pair] / total for pair in PAIRS}
    
    p_c_given_W = {}
    for pair in PAIRS:
        tc = target_conditioned_counts[pair]
        sz = paired_counts[pair]
        p_c_given_W[pair] = {c: (tc.get(c, 0) + 0.5 * alpha) / (sz + alpha * n / 12) for c in vocab}
    
    U, S, Vt = np.linalg.svd(PMI, full_matrices=False)
    sqrt_S = np.sqrt(np.abs(S[:rank])) * np.sign(S[:rank])
    W = (U[:, :rank] * sqrt_S).T
    C = (Vt[:rank, :].T * sqrt_S).T
    C_dag = np.linalg.pinv(C.T)
    
    return PMI, p_cw, p_w, p_c_given_W, p_W, W, C_dag, word2idx

**Verification**

In [4]:
def verify(PMI, p_cw, p_w, p_c_given_W, p_W, W, C_dag, word2idx):
    n = len(word2idx)
    idx = word2idx
    vocab = {i: w for w, i in idx.items()}
    log = lambda x: np.log(np.maximum(x, 1e-15))
    
    p_given_c = {w: p_cw[:, i] / p_w[i] for w, i in idx.items()}
    
    PAIRS = [('<royalty>', 'man'), ('<royalty>', 'woman')]
    p_cW = {pair: np.array([p_c_given_W[pair][vocab[i]] for i in range(n)]) for pair in PAIRS}
    p_Wc = {pair: p_cW[pair] * p_W[pair] / p_w for pair in PAIRS}
    
    CASES = [('king', 'man', '<royalty>', ('<royalty>', 'man')),
             ('queen', 'woman', '<royalty>', ('<royalty>', 'woman'))]
    
    for space_name, vecs, project, s in [("PMI space", PMI, lambda x: x, "I"),
                                         ("Embedding space", W, lambda x: C_dag @ x, "C†")]:
        print(f"\n{space_name}")
        print("=" * 50)
        for target, w1, w2, pair in CASES:
            obs = vecs[:, idx[target]] - vecs[:, idx[w1]] - vecs[:, idx[w2]]
            rho = log(p_given_c[target]) - log(p_cW[pair])
            p_w1c = np.array([p_given_c[c][idx[w1]] for c in idx])
            p_w2c = np.array([p_given_c[c][idx[w2]] for c in idx])
            sigma = log(p_Wc[pair]) - log(p_w1c) - log(p_w2c)
            tau = log(p_W[pair]) - log(p_w[idx[w1]]) - log(p_w[idx[w2]])
            residual = np.linalg.norm(obs - project(rho + sigma - tau))
            print(f"{target} = {w1} + {w2}")
            print(f"  ||ρ||={np.linalg.norm(rho):.2f}, ||σ||={np.linalg.norm(sigma):.2f}, |τ|={np.abs(tau):.4f}")
            print(f"  residual: {residual:.6f}")
    
    print("\n" + "=" * 50)
    print("ANALOGY: king - man + woman → ?")
    analogy = W[:, idx['king']] - W[:, idx['man']] + W[:, idx['woman']]
    dists = [np.linalg.norm(analogy - W[:, i]) for i in range(n)]
    for r, i in enumerate(np.argsort(dists)[:5]):
        print(f"  {r+1}. {vocab[i]:<12} dist={dists[i]:.4f}")

In [5]:
N = 1_000_000
counts, paired_counts, target_conditioned_counts = generate_data(n_windows=N, seed=42, scale=10)
PMI, p_cw, p_w, p_c_given_W, p_W, W, C_dag, word2idx = build_all(
    counts, paired_counts, target_conditioned_counts, N, rank=5
)
verify(PMI, p_cw, p_w, p_c_given_W, p_W, W, C_dag, word2idx)


PMI space
king = man + <royalty>
  ||ρ||=0.14, ||σ||=17.31, |τ|=0.0020
  residual: 0.000000
queen = woman + <royalty>
  ||ρ||=0.14, ||σ||=17.31, |τ|=0.0030
  residual: 0.000000

Embedding space
king = man + <royalty>
  ||ρ||=0.14, ||σ||=17.31, |τ|=0.0020
  residual: 0.000000
queen = woman + <royalty>
  ||ρ||=0.14, ||σ||=17.31, |τ|=0.0030
  residual: 0.000000

ANALOGY: king - man + woman → ?
  1. queen        dist=0.0382
  2. <royalty>    dist=3.1992
  3. royalty_17   dist=4.1569
  4. royalty_16   dist=4.1580
  5. royalty_19   dist=4.1598


**Results:**

With proper construction:
- $\|\rho\| \approx 0$ (paraphrase holds)
- $|\tau| \approx 0$ (marginal independence)
- Residual $= 0$ (Lemma 1 verified exactly)
- Analogy "king - man + woman" → queen ranks #1

Note, $\sigma$ isn't zero and it is very hard to make it so. Try to invent custom sampling data with $\sigma \approx 0$ to understand why 
it is so hard :)

PS/ $x$ doesn't actually have to lead $\tau = 0$ - thanks to symmetry in our data
we would still have residual even if $\tau \neq 0$ (we would only have some difficulties with
smoothing and making $\rho \approx 0$).

## Notes on Constants

### Why x = 1/6?

We chose x = 1/6 to make τ = 0 (marginal independence: p(man, royalty) = p(man)·p(royalty)).

Let p = 5x be the probability of sampling a single target. Then:
- p(man) = p/5 + (1-p)/2
- p(royalty) = p/5 + (1-p)
- p(man, royalty) = (1-p)/2

Solving p(man)·p(royalty) = p(man, royalty) gives x = 1/6.

Note: This choice is convenient but not essential. Due to symmetry in our construction (man/woman and king/queen are parallel), we have τ_king ≈ τ_queen, so their difference cancels in the analogy regardless of the individual τ values.

### Smoothing Constants

Laplace smoothing (α) prevents log(0) but must be applied consistently. The issue: p(c|w) from the co-occurrence matrix and p(c|W) from triple counts use different normalizations.

For unseen contexts with smoothing α:
- From co-occurrence: p(c|w) ∝ α / (N + αn)
- From triple counts: p(c|W) ∝ α / (|W|·N/2 + αn)

These differ by a factor related to the paraphrase frequency. To match scales, we use 0.5α for p(c|W):
```
p(c|W) = (count + 0.5α) / (pair_count + αn/12)
```

Using identical smoothing for both inflates the residual — many tokens accumulate small biases that sum to a significant error.